# Pandas II

## More indexing tricks

We'll start out with some data from Beer Advocate (see [Tom Augspurger](https://github.com/TomAugspurger/pydata-chi-h2t/blob/master/3-Indexing.ipynb) for some cool details on how he extracted this data)

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10

In [ ]:
df = pd.read_csv('data/beer_subset.csv.gz', parse_dates=['time'], compression='gzip')
df.head()

### Boolean indexing

Like a where clause in SQL. 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
df.abv < 5

In [ ]:
df.loc[df.abv < 5].head()

In [ ]:
df.loc[((df.abv < 5) & (df.time > pd.Timestamp('2009-06'))) | (df.review_overall >= 4.5)].head()

Be careful with the order of operations...

In [ ]:
2 > 1 & 0

Safest to use parentheses...

In [ ]:
(2 > 1) & 0

Select just the rows where the `beer_style` contains `'IPA'`:

In [ ]:
df.beer_style.str?

In [ ]:
df.beer_style.str.

In [ ]:
df.beer_style.str.contains('IPA')

In [ ]:
df.loc[df.beer_style.str.contains('IPA')].head()

Find the rows where the beer style is either `'American IPA'` or `'Pilsner'`:

In [ ]:
df[(df.beer_style == 'American IPA') | (df.beer_style == 'Pilsner')].head()

Or more succinctly:

In [ ]:
df[df.beer_style.isin(['American IPA', 'Pilsner'])].head()

#### Mini Exercise

- Select the rows where the scores of the 5 review_cols ('review_appearance', 'review_aroma', 'review_overall', 'review_palate', 'review_taste') are all at least 4.0.

- _Hint_: Like NumPy arrays, DataFrames have an any and all methods that check whether it contains any or all True values. These methods also take an axis argument for the dimension to remove.
    - 0 or 'index' removes (or aggregates over) the vertical dimension
    - 1 or 'columns' removes (aggregates over) the horizontal dimension.

In [ ]:
df.columns

In [ ]:
review_cols = [c for c in df.columns if c[0:6] == 'review']
review_cols

In [ ]:
df[(df.review_appearance >= 4) &
   (df.review_aroma >= 4) &
   (df.review_overall >= 4) &
   (df.review_palate >= 4) &
   (df.review_taste >= 4)].head()

Or the short way:

In [ ]:
df[(df[review_cols] >= 4).all(1)].head()

In [ ]:
(df[review_cols] >= 4)

In [ ]:
(df[review_cols] >= 4).all(axis=1)

In [ ]:
df[(df[review_cols] >= 4).all(1)].head()

Now select rows where the _average_ of the 5 `review_cols` is at least 4.

In [ ]:
df[review_cols].mean(axis=1)

In [ ]:
df[df[review_cols].mean(axis=1) >= 4].head()

## Hierarchical Indexing

- One of the most powerful and most complicated features of pandas
- Let's you represent high-dimensional datasets in a table

In [ ]:
reviews = df.set_index(['profile_name', 'beer_id', 'time'])
reviews.head()

In [ ]:
reviews = reviews.sort_index()
reviews.head()

In [ ]:
reviews.loc['05Harley']

In [ ]:
reviews.loc['05Harley',1307]

In [ ]:
reviews.loc[:,1307]

### Top Reviewers

Let's select all the reviews by the top reviewers, by label.

In [ ]:
top_reviewers = df['profile_name'].value_counts().head(5).index
top_reviewers

In [ ]:
reviews.loc[top_reviewers, :, :].head()

The syntax is a bit trickier when you want to specify a row Indexer *and* a column Indexer:

In [ ]:
reviews.loc[(top_reviewers, 99, :), ['beer_name', 'brewer_name']]

In [ ]:
reviews.loc[pd.IndexSlice[top_reviewers, 99, :], ['beer_name', 'brewer_id']]

Use `.loc` to select the `beer_name` and `beer_style` for the 10 most popular beers, as measured by number of reviews:

In [ ]:
top_beers = df['beer_id'].value_counts().head(10).index
top_beers

In [ ]:
reviews.loc[pd.IndexSlice[:, top_beers], ['beer_name', 'beer_style']]

### Beware "chained indexing"

You can sometimes get away with using `[...][...]`, but try to avoid it!

In [ ]:
df.loc[df.beer_style.str.contains('IPA')]['beer_name']

In [ ]:
df.loc[df.beer_style.str.contains('IPA')]['beer_name'] = 'yummy'

In [ ]:
df.loc[df.beer_style.str.contains('IPA')]['beer_name']

In [ ]:
df.loc[df.beer_style.str.contains('IPA'), 'beer_name'] = 'yummy'
df.loc[df.beer_style.str.contains('IPA')].head(2)

## Dates and Times

- Date and time data are inherently problematic
    - An unequal number of days in every month
    - An unequal number of days in a year (due to leap years)
    - Time zones that vary over space
    - etc
    
- The datetime built-in library handles temporal information down to the nanosecond

In [ ]:
from datetime import datetime

In [ ]:
now = datetime.now()
now

In [ ]:
now.day

In [ ]:
now.weekday()

Having a custom data type for dates and times is convenient because we can perform operations on them easily. 

For example, we may want to calculate the difference between two times:

In [ ]:
kfor = now - datetime(1985, 11, 21)
kfor

In [ ]:
'{:g}'.format(kfor.days // 365.25)

See [the docs](http://pandas.pydata.org/pandas-docs/stable/timeseries.html) for more information on Pandas' complex time and date functionalities...

## Example

In this section, we will manipulate data collected from ocean-going vessels on the eastern seaboard. Vessel operations are monitored using the Automatic Identification System (AIS), a safety at sea navigation technology which vessels are required to maintain and that uses transponders to transmit very high frequency (VHF) radio signals containing static information including ship name, call sign, and country of origin, as well as dynamic information unique to a particular voyage such as vessel location, heading, and speed. 

The International Maritime Organization’s (IMO) International Convention for the Safety of Life at Sea requires functioning AIS capabilities on all vessels 300 gross tons or greater and the US Coast Guard requires AIS on nearly all vessels sailing in U.S. waters. The Coast Guard has established a national network of AIS receivers that provides coverage of nearly all U.S. waters. AIS signals are transmitted several times each minute and the network is capable of handling thousands of reports per minute and updates as often as every two seconds. Therefore, a typical voyage in our study might include the transmission of hundreds or thousands of AIS encoded signals. This provides a rich source of spatial data that includes both spatial and temporal information.

For our purposes, we will use summarized data that describes the transit of a given vessel through a particular administrative area. The data includes the start and end time of the transit segment, as well as information about the speed of the vessel, how far it travelled, etc.

In [ ]:
segments = pd.read_csv("data/AIS/transit_segments.csv")
segments.head()

For example, we might be interested in the distribution of transit lengths, so we can plot them as a histogram:

In [ ]:
%matplotlib inline
segments.seg_length.hist(bins=500)

Though most of the transits appear to be short, there are a few longer distances that make the plot difficult to read. 

This is where a transformation is useful:

In [ ]:
segments.seg_length.apply(np.log).hist(bins=500)

We can see that although there are date/time fields in the dataset, they are not in any specialized format, such as `datetime`.

In [ ]:
segments.st_time.dtype

Our first order of business will be to convert these data to `datetime`. 

The `strptime` method parses a string representation of a date and/or time field, according to the expected format of this information.

In [ ]:
datetime.strptime(segments.st_time.ix[0], '%m/%d/%y %H:%M')

As a convenience, Pandas has a `to_datetime` method that will parse and convert an entire Series of formatted strings into `datetime` objects.

In [ ]:
pd.to_datetime(segments.st_time[:10])

Pandas also has a custom NA value for missing datetime objects, `NaT`.

In [ ]:
pd.to_datetime([None])

Finally, if `to_datetime()` has problems parsing any particular date/time format, you can pass the spec in using the `format=` argument.

## Merging and joining `DataFrame`s

In Pandas, we can combine tables according to the value of one or more *keys* that are used to identify rows, much like an index.

In [ ]:
df1 = pd.DataFrame(dict(id=range(4), age=np.random.randint(18, 31, size=4)))
df1

In [ ]:
df2 = pd.DataFrame(dict(id=list(range(3))+list(range(3)), 
                        score=np.random.random(size=6)))
df2

In [ ]:
pd.merge(df1, df2)

Notice that without any information about which column to use as a key, Pandas did the right thing and used the `id` column in both tables. Unless specified otherwise, `merge` will used any common column names as keys for merging the tables. 

Notice also that `id=3` from `df1` was omitted from the merged table. This is because, by default, `merge` performs an **inner join** on the tables, meaning that the merged table represents an intersection of the two tables.

In [ ]:
pd.merge(df1, df2, how='outer')

The **outer join** above yields the union of the two tables, so all rows are represented, with missing values inserted as appropriate. 

One can also perform **right** and **left** joins to include all rows of the right or left table (*i.e.* first or second argument to `merge`), but not necessarily the other.

### Back to the example

Now that we have the vessel transit information as we need it, we may want a little more information regarding the vessels themselves. 

In the `data/AIS` folder there is a second table that contains information about each of the ships that traveled the segments in the `segments` table.

In [ ]:
vessels = pd.read_csv("data/AIS/vessel_information.csv", index_col='mmsi')
vessels.head(2)

In [ ]:
segments.head(2)

We see that there is a `mmsi` value (a vessel identifier) in each table, but it is used as an index for the `vessels` table. In this case, we have to specify to join on the index for this table, and on the `mmsi` column for the other.

In [ ]:
segments_merged = pd.merge(vessels, segments, left_index=True, right_on='mmsi')

In [ ]:
segments_merged.head()

Notice that `mmsi` field that was an index on the `vessels` table is no longer an index on the merged table.

Each `DataFrame` also has a `.merge()` method that could have been used:

In [ ]:
vessels.merge(segments, left_index=True, right_on='mmsi').head()

Occasionally, there will be fields with the same in both tables that we do not wish to use to join the tables; they may contain different information, despite having the same name. 

In this case, Pandas will by default append suffixes `_x` and `_y` to the columns to uniquely identify them.

In [ ]:
segments['type'] = 'foo'
pd.merge(vessels, segments, left_index=True, right_on='mmsi').head()

This behavior can be overridden by specifying a `suffixes` argument, containing a list of the suffixes to be used for the columns of the left and right columns, respectively.

## Reshaping `DataFrame`s

This dataset in from Table 6.9 of [Statistical Methods for the Analysis of Repeated Measurements](http://www.amazon.com/Statistical-Methods-Analysis-Repeated-Measurements/dp/0387953701) by Charles S. Davis, pp. 161-163 (Springer, 2002). These data are from a multicenter, randomized controlled trial of botulinum toxin type B (BotB) in patients with cervical dystonia from nine U.S. sites.

* Randomized to placebo (N=36), 5000 units of BotB (N=36), 10,000 units of BotB (N=37)
* Response variable: total score on Toronto Western Spasmodic Torticollis Rating Scale (TWSTRS), measuring severity, pain, and disability of cervical dystonia (high scores mean more impairment)
* TWSTRS measured at baseline (week 0) and weeks 2, 4, 8, 12, 16 after treatment began

In [ ]:
cdystonia = pd.read_csv("data/cdystonia.csv", index_col=None)
cdystonia.head()

This dataset includes repeated measurements of the same individuals (longitudinal data). Its possible to present such information in (at least) two ways: showing each repeated measurement in their own row, or in multiple columns representing multiple measurements.

`.stack()` rotates the data frame so that columns are represented in rows:

In [ ]:
stacked = cdystonia.stack()
stacked

And there's a corresponding `.unstack()` which pivots back into columns:

In [ ]:
stacked.unstack().head()

For this dataset, it makes sense to create a hierarchical index based on the patient and observation:

In [ ]:
cdystonia2 = cdystonia.set_index(['patient','obs'])
cdystonia2.head()

In [ ]:
print(cdystonia2.index.levels[0])
print(cdystonia2.index.levels[1])

In [ ]:
cdystonia2.index.labels

In [ ]:
cdystonia2.index.values[0:5]

In [ ]:
cdystonia2.index.get_level_values('obs')

In [ ]:
cdystonia2.index.is_unique

If we want to transform this data so that repeated measurements are in columns, we can `unstack` the `twstrs` measurements according to `obs`:

In [ ]:
cdystonia2.head()

In [ ]:
twstrs_wide = cdystonia2['twstrs'].unstack('obs')
twstrs_wide.head()

And if we want to keep the other variables:

In [ ]:
cdystonia_wide = (cdystonia[['patient','site','id','treat','age','sex']]
                  .drop_duplicates()
                  .merge(twstrs_wide, right_index=True, left_on='patient', how='inner')
                  .head())
cdystonia_wide

Or to simplify things, we can set the patient-level information as an index before unstacking:

In [ ]:
(cdystonia.set_index(['patient','site','id','treat','age','sex','week'])['twstrs']
     .unstack('week').head())

### [`.melt()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

- To convert our "wide" format back to long, we can use the `melt` function. 
- This function is useful for `DataFrame`s where one or more columns are identifier variables (`id_vars`), with the remaining columns being measured variables (`value_vars`). 
- The measured variables are "unpivoted" to the row axis, leaving just two non-identifier columns, a *variable* and its corresponding *value*, which can both be renamed using optional arguments.

In [ ]:
pd.melt(cdystonia_wide, id_vars=['patient','site','id','treat','age','sex'], 
        var_name='obs', value_name='twsters').head()

## Pivoting

The `pivot` method allows a DataFrame to be transformed easily between long and wide formats in the same way as a pivot table is created in a spreadsheet. 

It takes three arguments: `index`, `columns` and `values`, corresponding to the DataFrame index (the row headers), columns and cell values, respectively.

For example, we may want the `twstrs` variable (the response variable) in wide format according to patient, as we saw with the unstacking method above:

In [ ]:
cdystonia.pivot(index='patient', columns='obs', values='twstrs').head()

If we omit the `values` argument, we get a `DataFrame` with hierarchical columns, just as when we applied `unstack` to the hierarchically-indexed table:

In [ ]:
cdystonia.pivot('patient', 'obs')

A related method, `pivot_table`, creates a spreadsheet-like table with a hierarchical index, and allows the values of the table to be populated using an arbitrary aggregation function.

In [ ]:
cdystonia.head()

In [ ]:
cdystonia.pivot_table(index=['site', 'treat'], columns='week', values='twstrs', 
                      aggfunc=max).head(20)

## Crosstabs and Summaries

For a simple cross-tabulation of group frequencies, the `crosstab` function (not a method) aggregates counts of data according to factors in rows and columns. The factors may be hierarchical if desired.

In [ ]:
pd.crosstab(cdystonia.sex, cdystonia.site)

And the `.describe()` method gives some useful summary information on the `DataFrame`:

In [ ]:
cdystonia.describe()

## Exercise 4

Open up [Lecture 4/Exercise 4.ipynb](./Exercise 4.ipynb) in your Jupyter notebook server.

Solutions are at [Lecture 4/Exercise 4 - Solutions.ipynb](./Exercise 4 - Solutions.ipynb)

## References

Slide materials inspired by and adapted from [Chris Fonnesbeck](https://github.com/fonnesbeck/statistical-analysis-python-tutorial) and [Tom Augspurger](https://github.com/TomAugspurger/pydata-chi-h2t)